In [71]:
# Dependencies and Setup
from datetime import datetime
import pandas as pd
import numpy as np
import os
import requests
import json
from pprint import pprint
from dotenv import load_dotenv

In [72]:
# API Call
load_dotenv()
API_KEY = os.getenv("Yelp_API") # put your key here if you don't have a .env file in your main project folder.
headers = {'Authorization': 'Bearer %s' % API_KEY}
base_url = "https://api.yelp.com/v3/businesses/search"

In [73]:
# Output Folder
today = datetime.today()
weekday = today.isoweekday()

days_week_dict = {1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday",
                 5: "Friday", 6: "Saturday", 7: "Sunday"}
day_of_the_week = days_week_dict.get(weekday)

outputFolder=f'{today.day}_{today.month}_{today.year}_{today.hour}h{today.minute}m'

In [30]:
def createFolder(outFolder):
    # Creates directory for current day of the week, raises an error if already exists.
    parent_dir = 'json_files'
    # output_dir = f'{day_of_the_week}1000'
    path = os.path.join(parent_dir, outFolder)
    try:
        os.mkdir(path)
    except:
        print('Folder already exists.')
        
    return path

In [35]:
path=createFolder(outputFolder)

Folder already exists.


In [18]:
stadium_data_df = pd.read_csv('Resources/stadium_coordinates.csv', dtype={
    "NFL Team": "string",
    "City": "string",
    "State": "string",
    "Stadium Name": "string",
    "Latitude": np.float64,
    "Longitude": np.float64,
    "Zip Code": "string"
})
stadium_data_df = stadium_data_df.dropna()
stadium_data_df

,NFL Team,City,State,Stadium Name,Latitude,Longitude,Zip Code
0,Arizona Cardinals,Glendale,Arizona,State Farm Stadium,33.528000,-112.263000,85305
1,Atlanta Falcons,Atlanta,Georgia,Mercedes-Benz Stadium,33.755556,-84.400000,30313
2,Baltimore Ravens,Baltimore,Maryland,M&T Bank Stadium,39.278056,-76.622778,21230
3,Buffalo Bills,Orchard Park,New York,Bills Stadium,42.774000,-78.787000,14127
4,Carolina Panthers,Charlotte,North Carolina,Bank of America Stadium,35.225833,-80.852778,28202
5,Chicago Bears,Chicago,Illinois,Soldier Field,41.862300,-87.616700,60605
6,Cincinnati Bengals,Cincinatti,Ohio,Paul Brown Stadium,39.095000,-84.516000,45202
7,Cleveland Browns,Cleveland,Ohio,FirstEnergy Stadium,41.506111,-81.699444,44114
8,Dallas Cowboys,Arlington,Texas,AT&T Stadium,32.747778,-97.092778,76011
9,Denver Broncos,Denver,Colorado,Empower Field at Mile High,39.741000,-105.019700,80204


In [25]:
def teamLoc(df, teamliga):
    zipi=zip(df["Latitude"], stadium_data_df["Longitude"], stadium_data_df[teamliga])
    return list(zipi)

In [34]:
stadium=teamLoc(stadium_data_df, 'NFL Team')

### API Parameters
search_limit = 20 #limit to 20 business names per query <br>
radius = 3000 #radius in meters <br>
term = "restaurants" <br>
search_params = {"term": term, "limit": search_limit, "radius": radius, "sort_by": "rating"}


In [36]:
# Function to download the JSON files
def downloadJSON(lista, term, radius, path):
    count_f = 0
    # API parameters
    search_params = {"term": term, "limit": 20, "radius": radius, "sort_by": "rating"}
    class Exception_1(Exception):
        """ Raised if "total" is not found in the request """
        pass
    for latitude, longitude, team in lista:

        search_params.update({"latitude": latitude, "longitude": longitude})
        response = requests.get(url = base_url, params = search_params, headers = headers).json()
        try:

            if response['total']== False:
                raise Exception_1
            elif response['total'] >= 1000:
                total = 1000
            else:
                total = response['total']

            for search_offset in range(0, total, 20):

                search_params.update({"offset": search_offset})
                response2 = requests.get(url = base_url, params = search_params, headers = headers).json()
                file_name = f'yelp_response_{team}_{search_offset}.json'
                output_path = os.path.join(path, file_name)

                with open(output_path,'w', encoding = 'utf-8') as f:
                    json.dump(response2, f, ensure_ascii=False, indent =4)
                f.close()

        except Exception_1 as e:
            print("Error: 'total' not found.\nPlease try again later.")
            print(e)

        print(f'{count_f}, {team} total: {total}')

        count_f += 1

In [37]:
downloadJSON(stadium, "restaurants", 3000, path)

0, Arizona Cardinals total: 101
1, Atlanta Falcons total: 665
2, Baltimore Ravens total: 784
3, Buffalo Bills total: 64
4, Carolina Panthers total: 448
5, Chicago Bears total: 1000
6, Cincinnati Bengals total: 425
7, Cleveland Browns total: 376
8, Dallas Cowboys total: 223
9, Denver Broncos total: 715
10, Detroit Lions total: 529
11, Green Bay Packers total: 124
12, Houston Texans total: 189
13, Indianapolis Colts total: 363
14, Jacksonville Jaguars total: 167
15, Kansas City Chiefs total: 29
16, Las Vegas Raiders total: 696
17, Los Angeles Chargers total: 337
18, Los Angeles Rams total: 337
19, Miami Dolphins total: 87
20, Minnesota Vikings total: 778
21, New England Patriots total: 44
22, New Orleans Saints total: 926
23, New York Giants total: 170
24, New York Jets total: 170
25, Philadelphia Eagles total: 281
26, Pittsburgh Steelers total: 460
27, San Francisco 49ers total: 223
28, Seattle Seahawks total: 1000
29, Tampa Bay Buccaners total: 267
30, Tennessee Titans total: 623
31, W

## Check json files integrity
Sometimes, due to an internal error in Yelp, the YELP API would return an empty file with an error code in it. 

Print to terminal if the json file has errors or not. 

If there are files with errors, the above request data needs to be re-run.

In [43]:
def JSONcheck(jsonpath):
    files_list = os.listdir(jsonpath)
    remove_list = []
    for file in files_list:
        open_path = os.path.join(jsonpath, file)
        with open(open_path, 'r', encoding = 'utf-8') as f:
            contents = json.loads(f.read())
            try:
                total = contents['total']
            except:
                print(f"INTERNAL_ERROR in {file}")
                print("Something went wrong internally, please try downloading the json file again later.\n")
                file = f'{file}'
                remove_list.append(file)
            else:
                print(f'\nNo errors found in the json file:\n{file}\n')
        
    if not remove_list:
        print(f'No erros in json files.\nNo files to be removed for the analysis.')
    else:
        print(f'Files with errors:\n{remove_list}')
    return remove_list

In [44]:
error_list = JSONcheck(path)


No errors found in the json file:
yelp_response_Arizona Cardinals_0.json


No errors found in the json file:
yelp_response_Arizona Cardinals_100.json


No errors found in the json file:
yelp_response_Arizona Cardinals_20.json


No errors found in the json file:
yelp_response_Arizona Cardinals_40.json


No errors found in the json file:
yelp_response_Arizona Cardinals_60.json


No errors found in the json file:
yelp_response_Arizona Cardinals_80.json


No errors found in the json file:
yelp_response_Atlanta Falcons_0.json


No errors found in the json file:
yelp_response_Atlanta Falcons_100.json


No errors found in the json file:
yelp_response_Atlanta Falcons_120.json


No errors found in the json file:
yelp_response_Atlanta Falcons_140.json


No errors found in the json file:
yelp_response_Atlanta Falcons_160.json


No errors found in the json file:
yelp_response_Atlanta Falcons_180.json


No errors found in the json file:
yelp_response_Atlanta Falcons_20.json


No errors found in th


No errors found in the json file:
yelp_response_Jacksonville Jaguars_120.json


No errors found in the json file:
yelp_response_Jacksonville Jaguars_140.json


No errors found in the json file:
yelp_response_Jacksonville Jaguars_160.json


No errors found in the json file:
yelp_response_Jacksonville Jaguars_20.json


No errors found in the json file:
yelp_response_Jacksonville Jaguars_40.json


No errors found in the json file:
yelp_response_Jacksonville Jaguars_60.json


No errors found in the json file:
yelp_response_Jacksonville Jaguars_80.json


No errors found in the json file:
yelp_response_Kansas City Chiefs_0.json


No errors found in the json file:
yelp_response_Kansas City Chiefs_20.json


No errors found in the json file:
yelp_response_Las Vegas Raiders_0.json


No errors found in the json file:
yelp_response_Las Vegas Raiders_100.json


No errors found in the json file:
yelp_response_Las Vegas Raiders_120.json


No errors found in the json file:
yelp_response_Las Vegas Rai


No errors found in the json file:
yelp_response_Tampa Bay Buccaners_260.json


No errors found in the json file:
yelp_response_Tampa Bay Buccaners_40.json


No errors found in the json file:
yelp_response_Tampa Bay Buccaners_60.json


No errors found in the json file:
yelp_response_Tampa Bay Buccaners_80.json


No errors found in the json file:
yelp_response_Tennessee Titans_0.json


No errors found in the json file:
yelp_response_Tennessee Titans_100.json


No errors found in the json file:
yelp_response_Tennessee Titans_120.json


No errors found in the json file:
yelp_response_Tennessee Titans_140.json


No errors found in the json file:
yelp_response_Tennessee Titans_160.json


No errors found in the json file:
yelp_response_Tennessee Titans_180.json


No errors found in the json file:
yelp_response_Tennessee Titans_20.json


No errors found in the json file:
yelp_response_Tennessee Titans_200.json


No errors found in the json file:
yelp_response_Tennessee Titans_220.json


No er

In [45]:
error_list

['yelp_response_Atlanta Falcons_540.json',
 'yelp_response_Chicago Bears_420.json',
 'yelp_response_Chicago Bears_800.json',
 'yelp_response_Dallas Cowboys_140.json',
 'yelp_response_Indianapolis Colts_120.json',
 'yelp_response_Minnesota Vikings_760.json',
 'yelp_response_New Orleans Saints_680.json',
 'yelp_response_Pittsburgh Steelers_40.json']

In [57]:
error_list[0].split('_')[2]

'Atlanta Falcons'

In [58]:
team_errors=[]
for i in error_list:
    aux=i.split("_")[2]
    team_errors.append(aux)
team_errors

['Atlanta Falcons',
 'Chicago Bears',
 'Chicago Bears',
 'Dallas Cowboys',
 'Indianapolis Colts',
 'Minnesota Vikings',
 'New Orleans Saints',
 'Pittsburgh Steelers']

## Try to download again the cities with errors

In [64]:
newfolder=f'again_for_errors.{today.day}_{today.month}_{today.year}_{today.hour}h{today.minute}m'
newpath=createFolder(newfolder)

Folder already exists.


In [65]:
er_df=stadium_data_df[stadium_data_df["NFL Team"].isin(team_errors)]

In [66]:
er_df

,NFL Team,City,State,Stadium Name,Latitude,Longitude,Zip Code
1,Atlanta Falcons,Atlanta,Georgia,Mercedes-Benz Stadium,33.755556,-84.400000,30313
5,Chicago Bears,Chicago,Illinois,Soldier Field,41.862300,-87.616700,60605
8,Dallas Cowboys,Arlington,Texas,AT&T Stadium,32.747778,-97.092778,76011
13,Indianapolis Colts,Indianapolis,Indiana,Lucas Oil Stadium,39.760056,-86.163806,46225
20,Minnesota Vikings,Minneapolis,Minnessota,U.S. Bank Stadium,44.974000,-93.258000,55415
22,New Orleans Saints,New Orleans,Louisiana,Mercedes--Benz Superdome,29.950833,-90.081111,70112
26,Pittsburgh Steelers,Pittsburgh,Pennsylvania,Heinz Field,40.446667,-80.015833,15212


In [67]:
new_list=teamLoc(er_df, "NFL Team" )	

In [69]:
downloadJSON(new_list, "restaurants", 3000, newpath)

0, Arizona Cardinals total: 1
Error: 'total' not found.
Please try again later.

1, Atlanta Falcons total: 1
Error: 'total' not found.
Please try again later.

2, Baltimore Ravens total: 1
Error: 'total' not found.
Please try again later.

3, Buffalo Bills total: 1
Error: 'total' not found.
Please try again later.

4, Carolina Panthers total: 1
Error: 'total' not found.
Please try again later.

5, Chicago Bears total: 1
6, Cincinnati Bengals total: 1


In [70]:
new_list

[(33.755556, -112.26299999999999, 'Arizona Cardinals'),
 (41.8623, -84.4, 'Atlanta Falcons'),
 (32.747778000000004, -76.622778, 'Baltimore Ravens'),
 (39.760056, -78.78699999999999, 'Buffalo Bills'),
 (44.974, -80.852778, 'Carolina Panthers'),
 (29.950833000000003, -87.6167, 'Chicago Bears'),
 (40.446667, -84.516, 'Cincinnati Bengals')]